In [26]:
import pandas as pd
df = pd.read_csv('parenti_totale.csv')

In [92]:
import re
def extract_aff(total_string):
    aff_list = []
    aff = re.findall("(?s)affil std {(.*?)}", total_string)
    for el in aff:
        aff_list.append(el)
    if len(aff)>0 :
        return aff_list
     
    aff2 = re.findall("(?s)affil str \"(.*?)\"", total_string)
    for el in aff2:
        aff_list.append(el)


    aff3 = re.findall("(?s)affil \"(.*?)\"", total_string)
    for el in aff3:
        aff_list.append(el)    
    
    return aff_list

def extract_country2(affiliations):
            countries = []
            for el in affiliations:
                country = re.findall("(?s)affil \"(.*?)\"", el)
                for el in country:
                    countries.append(el)
            if len(countries)>0 :
                return countries
            else:
                 for el in affiliations:
                    country = re.findall("(?s)affil str \"(.*?)\"", el)
                    for el in country:
                        countries.append(el)
            if len(countries)>0 :
                return countries            
            else:
                return affiliations


def extract_country(affiliations):
            countries = []
            for el in affiliations:
                country = re.findall("(?s)affil str \"(.*?)\"", el)
                for el in country:
                    countries.append(clean_long(el))


            if(len(countries)==0):
                for el in affiliations:
                    country = re.search("(?s)country \"(.*?)\"", el)
                    if(country != None):
                        
                        countries.append(extract_tra_virgolette(el[int(country.span()[0]):int(country.span()[1])]))
                    else:
                        countries.append("no country")            
            return countries

def remove_digits(row):
    countries = row['country']
    for el in countries:
        regex = re.findall("\d", el)
        if regex:
            countries.remove(el)
    return countries
 
     

def extract_first(countries):
    if countries:
        return countries[0]
    else:
        return "empty_list" 

def extract_last(countries):
    if countries:
        return countries[-1]
    else:
        return "empty_list"  
def countries_fl(riga):
    lista = []
    lista.append(riga['first_country'])
    lista.append(riga['last_country'])
    return lista                      

def clean_long(long_string):
    if len(long_string)>50:
        result = long_string.rsplit(None, 1)[-1]
        return result[:-1]        
    else:
        return long_string

def extract_tra_virgolette(stringa):
     country = re.search("(?s)\"(.*?)\"", stringa)
     return stringa[int(country.span()[0]+1):int(country.span()[1]-1)]

def controllo_uguali(row):
    if row['first_country'] == row['last_country']:
        return "uguali"
    else:
        return "diversi"

def len_affiliations(row):
    return len(row['country'])


In [93]:
df['affiliations'] = df.apply(lambda row: extract_aff(row['total_string']),axis=1)
df['affiliations']

0        [\n              affil "University of Sao Paul...
1        [\n              affil "University of Sao Paul...
2        [\n              affil "University of Sao Paul...
3        [\n              affil "Hebei Normal Universit...
4        [\n              affil "Hebei Normal Universit...
                               ...                        
37173    [\n              affil "National Veterinary Re...
37174    [\n              affil "Konkuk University",\n ...
37175    [\n              affil "Konkuk University",\n ...
37176    [\n              affil "Konkuk University",\n ...
37177    [\n              affil "Konkuk University",\n ...
Name: affiliations, Length: 37178, dtype: object

In [95]:
df['first_affiliation'] = df.apply(lambda row: extract_first(row['affiliations']),axis=1)
df['first_affiliation']

0        University of Sao Paulo, School of Veterinary ...
1        University of Sao Paulo, School of Veterinary ...
2        University of Sao Paulo, School of Veterinary ...
3        Hebei Normal University of Scicine and Technology
4        Hebei Normal University of Scicine and Technology
                               ...                        
37173    National Veterinary Research and Quarantine Se...
37174                                    Konkuk University
37175                                    Konkuk University
37176                                    Konkuk University
37177                                    Konkuk University
Name: first_affiliation, Length: 37178, dtype: object

In [94]:
df['affiliations'] = df.apply(lambda row: extract_country2(row['affiliations']),axis=1)
df['affiliations']

0        [University of Sao Paulo, School of Veterinary...
1        [University of Sao Paulo, School of Veterinary...
2        [University of Sao Paulo, School of Veterinary...
3        [Hebei Normal University of Scicine and Techno...
4        [Hebei Normal University of Scicine and Techno...
                               ...                        
37173    [National Veterinary Research and Quarantine S...
37174                                  [Konkuk University]
37175                                  [Konkuk University]
37176                                  [Konkuk University]
37177                                  [Konkuk University]
Name: affiliations, Length: 37178, dtype: object

In [98]:
df['first_affiliation'].value_counts()

One Health Institute & Karen C Drayer Wildlife Health\n Center, School of Veterinary Medicine, University of California, Davis       1424
China Animal Health and Epidemiology Center                                                                                           853
The University of Hong Kong                                                                                                           750
Istituto Zooprofilattico Sperimentale della Lombardia ed\n Emilia Romagna                                                             707
Harbin Veterinary Research Institute                                                                                                  693
                                                                                                                                     ... 
Southeast Poultry Research Laboratory, US National\n Poultry Research Center, Agricultural Research Service, USDA                       1
Contact:Shuichi Kubota Kyoto Biken

In [31]:
df['country'] = df.apply(lambda row: extract_country(row['affiliations']),axis=1)
df['country_clean'] = df.apply(lambda row: remove_digits(row),axis=1)

df['country_clean']

0             [Brasil]
1             [Brasil]
2             [Brasil]
3              [China]
4              [China]
             ...      
37173    [South Korea]
37174    [South Korea]
37175    [South Korea]
37176    [South Korea]
37177    [South Korea]
Name: country_clean, Length: 37178, dtype: object

In [32]:
df['first_country'] = df.apply(lambda row: extract_first(row['country']),axis=1)
df['last_country'] = df.apply(lambda row: extract_last(row['country']),axis=1)
df['Countries_FL'] = df.apply(lambda row: countries_fl(row),axis=1)


In [93]:
df['controllo'] = df.apply(lambda row: controllo_uguali(row),axis=1)
df['len'] = df.apply(lambda row: len_affiliations(row),axis=1)

In [35]:
df['first_country'].value_counts()

China                       7183
USA                         5059
P.R. China                  2271
Brazil                      1312
Italy                       1280
                            ... 
NOVAVAX INC [US]               1
INSTITUT                       1
gasperim@uw.edu                1
United States of America       1
qiongyuhao@cdrewu.edu          1
Name: first_country, Length: 356, dtype: int64

In [34]:
df

,Unnamed: 0,Unnamed: 0.1,total_string,affiliations,country,first_country,last_country,Countries_FL,controllo,len
0,0,0,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[\n affil ""University of Sao Paul...",[Brasil],Brasil,Brasil,"[Brasil, Brasil]",uguali,1
1,1,1,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[\n affil ""University of Sao Paul...",[Brasil],Brasil,Brasil,"[Brasil, Brasil]",uguali,1
2,2,2,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[\n affil ""University of Sao Paul...",[Brasil],Brasil,Brasil,"[Brasil, Brasil]",uguali,1
3,3,3,"Seq-entry ::= set {\n class pop-set,\n descr...","[\n affil ""Hebei Normal Universit...",[China],China,China,"[China, China]",uguali,1
4,4,4,"Seq-entry ::= set {\n class pop-set,\n descr...","[\n affil ""Hebei Normal Universit...",[China],China,China,"[China, China]",uguali,1
...,...,...,...,...,...,...,...,...,...,...
37173,994,994,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[\n affil ""National Veterinary Re...",[South Korea],South Korea,South Korea,"[South Korea, South Korea]",uguali,1
37174,995,995,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[\n affil ""Konkuk University"",\n ...",[South Korea],South Korea,South Korea,"[South Korea, South Korea]",uguali,1
37175,996,996,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[\n affil ""Konkuk University"",\n ...",[South Korea],South Korea,South Korea,"[South Korea, South Korea]",uguali,1
37176,997,997,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[\n affil ""Konkuk University"",\n ...",[South Korea],South Korea,South Korea,"[South Korea, South Korea]",uguali,1


In [66]:
df['first_country'].value_counts()

China                                              7183
USA                                                5059
P.R. China                                         2271
Brazil                                             1312
Italy                                              1280
                                                   ... 
CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS       1
[JP                                                   1
qiongyuhao@cdrewu.edu                                 1
Indi                                                  1
INSTITUT                                              1
Name: first_country, Length: 356, dtype: int64

In [99]:
A = df['first_affiliation'].value_counts()
A.to_csv("firstaffnolong234.csv",mode='a')

In [67]:
import pycountry
import time
cl=list(pycountry.countries)
countries_list=[]
count=-1
for paese in cl:
    count+=1
    try:
        name=list(pycountry.countries)[count].name
        countries_list.append(name)
    except AttributeError:
        time.sleep(1)
    try:
        common_name=list(pycountry.countries)[count].common_name
        countries_list.append(common_name)
    except AttributeError:
        time.sleep(1)
    try:
        official_name=list(pycountry.countries)[count].official_name
        countries_list.append(official_name)
    except AttributeError:
        time.sleep(1)
    try:
        alpha_2=list(pycountry.countries)[count].alpha_2
        countries_list.append(alpha_2)
    except AttributeError:
        time.sleep(1)
    try:
        alpha_3=list(pycountry.countries)[count].alpha_3
        countries_list.append(alpha_3)
    except AttributeError:
        time.sleep(1)

len(countries_list) #in questa lista ci sono dentro tutti i riferimenti possibili ai paesi

926

In [68]:
countries_list.append('UK') #manca UK, lo aggiungiamo
countries_list.append('PR China')
countries_list.append('England')
countries_list.append('Perù')
countries_list.append('México')
countries_list.append('España')
countries_list.append('United States of America')
countries_list.append('Brasil')
countries_list.append('Deutschland') 
countries_list.append('Iran')
countries_list.append('Russia')
countries_list.append('Republic of Korea')
countries_list.append('Italia')
countries_list.append('Korea')
countries_list.append('UNITED STATES')
countries_list.append('SPAIN')
countries_list.append('GERMANY')
countries_list.append('belgique')

In [69]:
ustates = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY",'New York (JBB)','New York (PD)',"Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

city=pd.read_csv("https://raw.githubusercontent.com/grammakov/USA-cities-and-states/master/us_cities_states_counties.csv",delimiter='|')
uscity=city['City'].values.tolist()
uscity=list(dict.fromkeys(uscity))
len(uscity)

ustates=ustates+uscity
len(ustates)


cities=pd.read_csv("/parenti_veri/world-cities_csv.csv",converters={'Affiliations': eval})
print(cities.head())

chinalist = cities[cities['country']=='China']['name'].tolist()
fralist = cities[cities['country']=='France']['name'].tolist()
spagnalist = cities[cities['country']=='Spain']['name'].tolist()
gerlist = cities[cities['country']=='Germany']['name'].tolist()
nedlist = cities[cities['country']=='Netherlands']['name'].tolist()
japlist = cities[cities['country']=='Japan']['name'].tolist()
italist = cities[cities['country']=='Italy']['name'].tolist()
uklist = cities[cities['country']=='United Kingdom']['name'].tolist()

               name               country          subcountry  geonameid
0      les Escaldes               Andorra  Escaldes-Engordany    3040051
1  Andorra la Vella               Andorra    Andorra la Vella    3041563
2    Umm al Qaywayn  United Arab Emirates      Umm al Qaywayn     290594
3    Ras al-Khaimah  United Arab Emirates     Raʼs al Khaymah     291074
4      Khawr Fakkān  United Arab Emirates        Ash Shāriqah     291696


In [70]:
p1 = df

In [71]:
#PROVIAMO A TENERE SEPRATI I VALORI DI INVESTIMENTO E AUTORE PRINCIPALE

diz_cov_country={}
id_nonaz=[]

for index,row in p1.iterrows():
    inout=0 #per scovare gli indici che non hanno paese ma solo laboratorio
    count=-1
    for el in row['Countries_FL']:
        count+=1
        if row['Countries_FL'][count] in countries_list:
            inout+=1
            if row['Countries_FL'][count] in diz_cov_country:
                diz_cov_country[row['Countries_FL'][count]][count]+=1
            else:
                diz_cov_country[row['Countries_FL'][count]]=[0,0]
                diz_cov_country[row['Countries_FL'][count]][count]+=1
        elif row['Countries_FL'][count] in ustates:
            inout+=1
            if 'USA' in diz_cov_country:
                diz_cov_country['USA'][count]+=1
            else:
                diz_cov_country['USA']=[0,0]
                diz_cov_country['USA'][count]+=1
                
        elif row['Countries_FL'][count] in chinalist:
                    inout+=1
                    if 'China' in diz_cov_country:
                        diz_cov_country['China'][count]+=1
                    else:
                        diz_cov_country['China']=[0,0]
                        diz_cov_country['China'][count]+=1
                
        elif row['Countries_FL'][count] in fralist:
                    inout+=1
                    if 'France' in diz_cov_country:
                        diz_cov_country['France'][count]+=1
                    else:
                        diz_cov_country['France']=[0,0]
                        diz_cov_country['France'][count]+=1
                
        elif row['Countries_FL'][count] in spagnalist:
                    inout+=1
                    if 'Spain' in diz_cov_country:
                        diz_cov_country['Spain'][count]+=1
                    else:
                        diz_cov_country['Spain']=[0,0]
                        diz_cov_country['Spain'][count]+=1
                    
        elif row['Countries_FL'][count] in gerlist:
                    inout+=1
                    if 'Germany' in diz_cov_country:
                        diz_cov_country['Germany'][count]+=1
                    else:
                        diz_cov_country['Germany']=[0,0]
                        diz_cov_country['Germany'][count]+=1
                        
        elif row['Countries_FL'][count] in nedlist:
                    inout+=1
                    if 'Netherlands' in diz_cov_country:
                        diz_cov_country['Netherlands'][count]+=1
                    else:
                        diz_cov_country['Netherlands']=[0,0]
                        diz_cov_country['Netherlands'][count]+=1
                        
        elif row['Countries_FL'][count] in japlist:
                    inout+=1
                    if 'Japan' in diz_cov_country:
                        diz_cov_country['Japan'][count]+=1
                    else:
                        diz_cov_country['Japan']=[0,0]
                        diz_cov_country['Japan'][count]+=1
                        
                        
        elif row['Countries_FL'][count] in italist:
                    inout+=1
                    if 'Italy' in diz_cov_country:
                        diz_cov_country['Italy'][count]+=1
                    else:
                        diz_cov_country['Italy']=[0,0]
                        diz_cov_country['Italy'][count]+=1
                        
                        
        elif row['Countries_FL'][count] in uklist:
                    inout+=1
                    if 'United Kingdom' in diz_cov_country:
                        diz_cov_country['United Kingdom'][count]+=1
                    else:
                        diz_cov_country['United Kingdom']=[0,0]
                        diz_cov_country['United Kingdom'][count]+=1
                
                

                
                
                
        else:
            spl= row['Countries_FL'][count].split(" ")
            for elemento in spl:
                if elemento in countries_list:
                    inout+=1
                    if elemento in diz_cov_country:
                        diz_cov_country[elemento][count]+=1
                    else:
                        diz_cov_country[elemento]=[0,0]
                        diz_cov_country[elemento][count]+=1
                        
                elif elemento in ustates:
                    inout+=1
                    if 'USA' in diz_cov_country:
                        diz_cov_country['USA'][count]+=1
                    else:
                        diz_cov_country['USA']=[0,0]
                        diz_cov_country['USA'][count]+=1
                        
                elif elemento in chinalist:
                    inout+=1
                    if 'China' in diz_cov_country:
                        diz_cov_country['China'][count]+=1
                    else:
                        diz_cov_country['China']=[0,0]
                        diz_cov_country['China'][count]+=1
                
                elif elemento in fralist:
                    inout+=1
                    if 'France' in diz_cov_country:
                        diz_cov_country['France'][count]+=1
                    else:
                        diz_cov_country['France']=[0,0]
                        diz_cov_country['France'][count]+=1
                
                elif elemento in spagnalist:
                    inout+=1
                    if 'Spain' in diz_cov_country:
                        diz_cov_country['Spain'][count]+=1
                    else:
                        diz_cov_country['Spain']=[0,0]
                        diz_cov_country['Spain'][count]+=1
                    
                elif elemento in gerlist:
                    inout+=1
                    if 'Germany' in diz_cov_country:
                        diz_cov_country['Germany'][count]+=1
                    else:
                        diz_cov_country['Germany']=[0,0]
                        diz_cov_country['Germany'][count]+=1
                        
                elif elemento in nedlist:
                    inout+=1
                    if 'Netherlands' in diz_cov_country:
                        diz_cov_country['Netherlands'][count]+=1
                    else:
                        diz_cov_country['Netherlands']=[0,0]
                        diz_cov_country['Netherlands'][count]+=1
                        
                elif elemento in japlist:
                    inout+=1
                    if 'Japan' in diz_cov_country:
                        diz_cov_country['Japan'][count]+=1
                    else:
                        diz_cov_country['Japan']=[0,0]
                        diz_cov_country['Japan'][count]+=1
                        
                        
                elif elemento in italist:
                    inout+=1
                    if 'Italy' in diz_cov_country:
                        diz_cov_country['Italy'][count]+=1
                    else:
                        diz_cov_country['Italy']=[0,0]
                        diz_cov_country['Italy'][count]+=1
                        
                        
                elif elemento in uklist:
                    inout+=1
                    if 'United Kingdom' in diz_cov_country:
                        diz_cov_country['United Kingdom'][count]+=1
                    else:
                        diz_cov_country['United Kingdom']=[0,0]
                        diz_cov_country['United Kingdom'][count]+=1
                
                

                
                
                else:
                    try:
                        dop=str(spl[0]+' '+spl[1])
                        if dop in countries_list:
                            inout+=1
                            if dop in diz_cov_country:
                                diz_cov_country[dop][count]+=1
                            else:
                                diz_cov_country[dop]=[0,0]
                                diz_cov_country[dop][count]+=1
                            
                    except IndexError:
                        inout+=0
                        
    if inout < 2:
        id_nonaz.append(index)

#round dei valori float per eccesso:
#for key in diz_cov_country:
#    diz_cov_country[key]=int(np.ceil(diz_cov_country[key]))   

#ordinamento del dizionario   
sort_diz=sorted(diz_cov_country.items(),key=lambda item: item[1], reverse=True)
sort_diz      

[('China', [9579, 9584]),
 ('USA', [6367, 6379]),
 ('Brazil', [1312, 1305]),
 ('Italy', [1280, 1275]),
 ('Thailand', [1058, 1058]),
 ('Korea', [788, 792]),
 ('NL', [620, 373]),
 ('US', [618, 386]),
 ('Germany', [586, 589]),
 ('Netherlands', [569, 570]),
 ('Hong Kong', [538, 541]),
 ('KR', [490, 490]),
 ('Japan', [419, 806]),
 ('Taiwan', [419, 418]),
 ('France', [414, 475]),
 ('Malaysia', [389, 389]),
 ('Egypt', [365, 372]),
 ('Sweden', [362, 365]),
 ('Canada', [347, 355]),
 ("People's Republic of China", [342, 342]),
 ('Australia', [287, 301]),
 ('UK', [278, 281]),
 ('Republic of Korea', [266, 266]),
 ('Iran', [258, 253]),
 ('India', [252, 249]),
 ('South Africa', [194, 194]),
 ('Luxembourg', [183, 226]),
 ('Slovenia', [158, 158]),
 ('Viet Nam', [147, 147]),
 ('Kingdom of Saudi Arabia', [145, 145]),
 ('Turkey', [141, 139]),
 ('FR', [138, 138]),
 ('Russia', [127, 116]),
 ('Singapore', [123, 121]),
 ('AND', [118, 118]),
 ('Saudi Arabia', [110, 108]),
 ('Uruguay', [106, 106]),
 ('Poland',

In [72]:
count=0
diz_aus={}
for key in diz_cov_country:
    
    if len(key) == 2:
        try:
            country = pycountry.countries.get(alpha_2=key)
            name=country.name
            if name in diz_aus:
                diz_aus[name]+=diz_cov_country[key]
            if name not in diz_aus:
                diz_aus[name]=diz_cov_country[key]

        except AttributeError:
            count+=1
            
    if len(key) == 3:
        try:
            country = pycountry.countries.get(alpha_3=key)
            name=country.name
            if name in diz_aus:
                diz_aus[name]+=diz_cov_country[key]
            if name not in diz_aus:
                diz_aus[name]=diz_cov_country[key]
                
        except AttributeError:
            count+=1

 
diz_aus 

{'United States': [6367, 6379, 618, 386],
 'Japan': [76, 76],
 'Netherlands': [620, 373],
 'Andorra': [118, 118],
 'Singapore': [33, 33],
 'China': [39, 39],
 'Morocco': [0, 7],
 'United Kingdom': [77, 36],
 'France': [138, 138],
 'Canada': [47, 0],
 'Cyprus': [94, 0],
 'Cuba': [14, 4],
 'Taiwan, Province of China': [16, 0],
 'Colombia': [13, 13],
 "Lao People's Democratic Republic": [54, 54],
 'Bouvet Island': [13, 13],
 'Italy': [7, 0],
 'Switzerland': [9, 0],
 'Germany': [1, 1],
 'Indonesia': [26, 26],
 'Saudi Arabia': [24, 24],
 'Spain': [21, 21],
 'Korea, Republic of': [490, 490]}

In [73]:
for key in diz_aus:
    count=-1
    pari=0
    disp=0
    if len(diz_aus[key]) > 2:
        for el in diz_aus[key]:
            count+=1
            if count==0 or count==2:
                pari+=diz_aus[key][count]
            if count==1 or count==3:
                disp+=diz_aus[key][count]
        diz_aus[key]=[pari,disp]
diz_aus

{'United States': [6985, 6765],
 'Japan': [76, 76],
 'Netherlands': [620, 373],
 'Andorra': [118, 118],
 'Singapore': [33, 33],
 'China': [39, 39],
 'Morocco': [0, 7],
 'United Kingdom': [77, 36],
 'France': [138, 138],
 'Canada': [47, 0],
 'Cyprus': [94, 0],
 'Cuba': [14, 4],
 'Taiwan, Province of China': [16, 0],
 'Colombia': [13, 13],
 "Lao People's Democratic Republic": [54, 54],
 'Bouvet Island': [13, 13],
 'Italy': [7, 0],
 'Switzerland': [9, 0],
 'Germany': [1, 1],
 'Indonesia': [26, 26],
 'Saudi Arabia': [24, 24],
 'Spain': [21, 21],
 'Korea, Republic of': [490, 490]}

In [74]:
dup=list(diz_cov_country.keys())
import collections
print([item for item, count in collections.Counter(dup).items() if count > 1])

[]


In [129]:
for key in diz_aus:
    for chiave in diz_cov_country:
        if key == chiave:
            diz_cov_country[chiave]+=diz_aus[key]
sort_diz_fin=sorted(diz_cov_country.items(),key=lambda item: item[0], reverse=True)

for key in diz_cov_country:
    count=-1
    pari=0
    disp=0
    if len(diz_cov_country[key]) > 2:
        for el in diz_cov_country[key]:
            count+=1
            if count==0 or count==2:
                pari+=diz_cov_country[key][count]
            if count==1 or count==3:
                disp+=diz_cov_country[key][count]
        diz_cov_country[key]=[pari,disp]
diz_cov_country

{'Brasil': [92, 84],
 'China': [9618, 9623],
 'Canada': [394, 355],
 'Egypt': [365, 372],
 'India': [252, 249],
 'France': [552, 613],
 'Korea': [788, 792],
 'USA': [6985, 6765],
 'United Kingdom': [150, 108],
 'Gabon': [21, 21],
 'Uruguay': [106, 106],
 'Thailand': [1058, 1058],
 'Republic of Korea': [266, 266],
 'Iran': [258, 253],
 'New Zealand': [10, 10],
 'Turkey': [141, 139],
 'Chile': [40, 40],
 'Czech Republic': [10, 10],
 'Germany': [587, 590],
 "People's Republic of China": [342, 342],
 'Malaysia': [389, 389],
 'United States of America': [1, 1],
 'Italy': [1287, 1275],
 'Taiwan': [419, 418],
 'Brazil': [1312, 1305],
 'UK': [278, 281],
 'US': [618, 386],
 'Luxembourg': [183, 226],
 'Slovenia': [158, 158],
 'Spain': [80, 77],
 'Argentina': [29, 29],
 'Israel': [24, 28],
 'Netherlands': [1189, 943],
 'JP': [76, 76],
 'NL': [620, 373],
 'AND': [118, 118],
 'SG': [33, 33],
 'CN': [39, 39],
 'Greece': [72, 69],
 'Sweden': [362, 365],
 'Russia': [127, 116],
 'Japan': [495, 882],
 '

In [75]:
sorted(diz_cov_country.items(),key=lambda item: item[1], reverse=True)

[('China', [9579, 9584]),
 ('USA', [6367, 6379, 618, 386]),
 ('Brazil', [1312, 1305]),
 ('Italy', [1280, 1275]),
 ('Thailand', [1058, 1058]),
 ('Korea', [788, 792]),
 ('NL', [620, 373]),
 ('US', [618, 386]),
 ('Germany', [586, 589]),
 ('Netherlands', [569, 570]),
 ('Hong Kong', [538, 541]),
 ('KR', [490, 490]),
 ('Japan', [419, 806]),
 ('Taiwan', [419, 418]),
 ('France', [414, 475]),
 ('Malaysia', [389, 389]),
 ('Egypt', [365, 372]),
 ('Sweden', [362, 365]),
 ('Canada', [347, 355]),
 ("People's Republic of China", [342, 342]),
 ('Australia', [287, 301]),
 ('UK', [278, 281]),
 ('Republic of Korea', [266, 266]),
 ('Iran', [258, 253]),
 ('India', [252, 249]),
 ('South Africa', [194, 194]),
 ('Luxembourg', [183, 226]),
 ('Slovenia', [158, 158]),
 ('Viet Nam', [147, 147]),
 ('Kingdom of Saudi Arabia', [145, 145]),
 ('Turkey', [141, 139]),
 ('FR', [138, 138]),
 ('Russia', [127, 116]),
 ('Singapore', [123, 121]),
 ('AND', [118, 118]),
 ('Saudi Arabia', [110, 108]),
 ('Uruguay', [106, 106]),
 

In [77]:
diz_cov_country['UnitedStates2']=diz_cov_country['United States']+diz_cov_country['United States of America']+diz_cov_country['USA']+diz_cov_country['US']
diz_cov_country['China2']=diz_cov_country['China'] + diz_cov_country['PR China'] + diz_cov_country['Hong Kong'] + diz_cov_country["People's Republic of China"] + diz_cov_country['Taiwan']
diz_cov_country['Italy2']=diz_cov_country['Italy'] + diz_cov_country['Italia'] + diz_cov_country['IT']
diz_cov_country['Netherlands2']=diz_cov_country['Netherlands'] + diz_cov_country['NL'] 
diz_cov_country['France2']=diz_cov_country['France'] + diz_cov_country['FR'] 
diz_cov_country['UnitedKingdom2']=diz_cov_country['UK'] + diz_cov_country['United Kingdom'] + diz_cov_country['England']
diz_cov_country['Brazil2']=diz_cov_country['Brazil'] + diz_cov_country['Brasil']
diz_cov_country['Iran2']=diz_cov_country['Islamic Republic of Iran']  + diz_cov_country['Iran']
diz_cov_country['SaudiArabia2']=diz_cov_country['Saudi Arabia'] + diz_cov_country['Kingdom of Saudi Arabia']
diz_cov_country['Korea2']=diz_cov_country['Republic of Korea'] + diz_cov_country['Korea']+ diz_cov_country['KR']

In [78]:
USA=['United States','United States of America','UNITED STATES','USA','US']
CHINA=['China','PR China','Hong Kong',"People's Republic of China",'Taiwan']
ITALIA=['Italy','Italia','IT']
NL=['Netherlands','NL']
FRANCE=['France','FR']
UK=['United Kingdom of Great Britain and Northern Ireland','United Kingdom','England', 'UK']
GERMANIA=['Germany','GERMANY','Deutschland']
BRASILE=['Brazil','Brasil']
IRAN=['Islamic Republic of Iran','Iran']
ARABIA_SAUDITA=['Saudi Arabia','Kingdom of Saudi Arabia']
KOREA=['Republic of Korea','Korea','KR']

#eliminazione chiavi vecchie non utili
for key, value in list(diz_cov_country.items()):
    if key in USA or key in CHINA or key in ITALIA or key in UK or key in GERMANIA or key in BRASILE or key in IRAN or key in ARABIA_SAUDITA or key in KOREA or key in NL or key in FRANCE: #poi gestisci manualmente UK
        del(diz_cov_country[key])
len(diz_cov_country)

89

In [79]:
diz_cov_country['United States'] = diz_cov_country.pop('UnitedStates2')
diz_cov_country['China'] = diz_cov_country.pop('China2')
diz_cov_country['Italy'] = diz_cov_country.pop('Italy2')
diz_cov_country['United Kingdom'] = diz_cov_country.pop('UnitedKingdom2')
diz_cov_country['Brazil'] = diz_cov_country.pop('Brazil2')
diz_cov_country['Iran'] = diz_cov_country.pop('Iran2')
diz_cov_country['Saudi Arabia'] = diz_cov_country.pop('SaudiArabia2')
diz_cov_country['Korea'] = diz_cov_country.pop('Korea2')
diz_cov_country['Netherlands'] = diz_cov_country.pop('Netherlands2')
diz_cov_country['France'] = diz_cov_country.pop('France2')

In [80]:
sorted_diz_fin=sorted(diz_cov_country.items(),key=lambda item: item[1], reverse=True)
sorted_diz_fin

[('China', [9579, 9584, 1, 1, 538, 541, 342, 342, 419, 418]),
 ('Brazil', [1312, 1305, 92, 84]),
 ('Italy', [1280, 1275, 4, 4, 7, 0]),
 ('Thailand', [1058, 1058]),
 ('Netherlands', [569, 570, 620, 373]),
 ('Japan', [419, 806]),
 ('France', [414, 475, 138, 138]),
 ('Malaysia', [389, 389]),
 ('Egypt', [365, 372]),
 ('Sweden', [362, 365]),
 ('Canada', [347, 355]),
 ('Australia', [287, 301]),
 ('United Kingdom', [278, 281, 73, 72, 3, 3]),
 ('Korea', [266, 266, 788, 792, 490, 490]),
 ('India', [252, 249]),
 ('South Africa', [194, 194]),
 ('Luxembourg', [183, 226]),
 ('Slovenia', [158, 158]),
 ('Viet Nam', [147, 147]),
 ('Turkey', [141, 139]),
 ('Russia', [127, 116]),
 ('Singapore', [123, 121]),
 ('AND', [118, 118]),
 ('Saudi Arabia', [110, 108, 145, 145]),
 ('Uruguay', [106, 106]),
 ('Poland', [97, 88]),
 ('CY', [94, 0]),
 ('Finland', [79, 79]),
 ('GB', [77, 36]),
 ('JP', [76, 76]),
 ('Kenya', [74, 74]),
 ('Greece', [72, 69]),
 ('Morocco', [71, 71]),
 ('United Arab Emirates', [66, 66]),
 ('

In [81]:
#ultimo filtro
#sistemazione dizionario ausiliario
for key in diz_cov_country:
    count=-1
    pari=0
    disp=0
    if len(diz_cov_country[key]) > 2:
        for el in diz_cov_country[key]:
            count+=1
            if count==0 or count==2 or count==4 or count==6 or count==8:
                pari+=diz_cov_country[key][count]
            if count==1 or count==3 or count==5 or count==7 or count==9:
                disp+=diz_cov_country[key][count]
        diz_cov_country[key]=[pari,disp]
diz_cov_country

{'Canada': [347, 355],
 'Egypt': [365, 372],
 'India': [252, 249],
 'Gabon': [21, 21],
 'Uruguay': [106, 106],
 'Thailand': [1058, 1058],
 'New Zealand': [10, 10],
 'Turkey': [141, 139],
 'Chile': [40, 40],
 'Czech Republic': [10, 10],
 'Malaysia': [389, 389],
 'Luxembourg': [183, 226],
 'Slovenia': [158, 158],
 'Spain': [59, 56],
 'Argentina': [29, 29],
 'Israel': [24, 28],
 'JP': [76, 76],
 'AND': [118, 118],
 'SG': [33, 33],
 'CN': [39, 39],
 'Greece': [72, 69],
 'Sweden': [362, 365],
 'Russia': [127, 116],
 'Japan': [419, 806],
 'MA': [0, 7],
 'Ireland': [16, 16],
 'GB': [77, 36],
 'Tunisia': [8, 8],
 'CA': [47, 0],
 'CY': [94, 0],
 'Australia': [287, 301],
 'CUB': [14, 4],
 'Cuba': [1, 10],
 'TW': [16, 0],
 'CO': [13, 13],
 'Singapore': [123, 121],
 'Korea, Republic of': [3, 3],
 'Switzerland': [10, 23],
 'Poland': [97, 88],
 'LA': [54, 54],
 'Hungary': [64, 63],
 'South Africa': [194, 194],
 'Costa Rica': [4, 4],
 'Indonesia': [12, 12],
 'Denmark': [10, 6],
 'Mexico': [55, 55],
 

In [82]:
listone=list(diz_cov_country.values())
chiavi=list(diz_cov_country.keys())
pa=[]
fun=[]
som=[]
listone
for li in listone:
    somma=0
    count=-1
    for el in li:
        count+=1
        if count==0:
            pa.append(el)
            somma+=el
        if count==1:
            fun.append(el)
            somma+=el
    som.append(somma)


countrycov_pub=pd.DataFrame(list(zip(chiavi,pa,fun,som)),columns=['Country','Principal_Author','Funding_country','Sum'])
countrycov_pub=countrycov_pub.sort_values(by=['Sum'],ascending=False).reset_index() 
countrycov_pub=countrycov_pub.drop('index',axis=1)
countrycov_pub=countrycov_pub[0:20]


#taglia india, brasile, iran, korea
#countrycov_pub=countrycov_pub.drop([4,8,10,14,17],axis=0).reset_index()
#countrycov_pub=countrycov_pub.drop('index',axis=1)
countrycov_pub

,Country,Principal_Author,Funding_country,Sum
0,China,10879,10886,21765
1,United States,7605,7152,14757
2,Korea,1544,1548,3092
3,Brazil,1404,1389,2793
4,Italy,1291,1279,2570
5,Netherlands,1189,943,2132
6,Thailand,1058,1058,2116
7,Japan,419,806,1225
8,France,552,613,1165
9,Malaysia,389,389,778


In [146]:
countrycov_pub.to_csv("paesi_vicini.csv")

In [144]:
# passo a lavoro su affiliations e centri di ricerca

df['first_aff'] = df.apply(lambda row: extract_first(row['affiliations']),axis=1)
df['first_aff'].value_counts()

In [145]:
A = df['first_aff'].value_counts()
A.to_csv("aff.csv",mode='a')

In [30]:
test = df
test = test.drop(['total_string'], axis=1)
test = test.drop(['affiliations'], axis=1)

test.to_csv("testfirstlat.csv")

In [44]:
def extract_date(total_string):
    date = re.findall("(?s)date(.*?)}", total_string) 
    return extract_year(date)
def fill_date(row):
    if not row['date']:
        date = re.findall("(?s)date(.*?)}", row['total_string']) 
        row['date'] = date
    else:
        row['date'] = row['date']

    return row['date']              
def extract_year(dates):
    years =[]
    for date in dates:
        year = re.findall("(?s)name \"(.*?)\"", date)
        years.append(year)
    return years
def clean_empty(dates):
    new_list = list(filter(None, dates))
    return new_list
 
def first_date(date):
    if date:
        data0 = str(date[0])
        data = data0[2:-2]

        if len(data) == 4:
            return data
        else:
            return data[-4:]       
    else:
        return "no_data"
def check_wrongformat(first_date):
     
     fd = first_date[2:-2]
     x = re.search('([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9])',fd)
     if x:
        return first_date[2:-2]
     else:
         return "wrong format"

In [45]:
df['clean_date'] = df.apply(lambda row: clean_empty(row['date']),axis=1)

In [46]:
df['first_date'] = df.apply(lambda row: first_date(row['clean_date']),axis=1)

In [47]:
df['first_date']

0        2018
1        2018
2        2018
3        2018
4        2018
         ... 
37173    1986
37174    2004
37175    2003
37176    2003
37177    2003
Name: first_date, Length: 37178, dtype: object

In [53]:
def extract_taxname(total_string):
    taxname = re.findall("(?s)taxname \"(.*?)\"", total_string) 
    return taxname

def first_taxname(date):
    if date:
        return str(date[0])
    else:
        return "no_taxname"    

In [54]:
df['first_taxname'] = df.apply(lambda row: first_taxname(row['taxname']),axis=1)

df['first_taxname']

0                     Alphacoronavirus 1
1                     Alphacoronavirus 1
2                     Alphacoronavirus 1
3        Porcine epidemic diarrhea virus
4        Porcine epidemic diarrhea virus
                      ...               
37173        Infectious bronchitis virus
37174        Infectious bronchitis virus
37175        Infectious bronchitis virus
37176        Infectious bronchitis virus
37177        Infectious bronchitis virus
Name: first_taxname, Length: 37178, dtype: object

In [55]:
df['first_taxname'].value_counts()

Infectious bronchitis virus                           8165
Porcine epidemic diarrhea virus                       5349
synthetic construct                                   2239
Feline coronavirus                                    2190
no_taxname                                            1804
                                                      ... 
Coronavirus RCoV/RMU10_3052/Apo_fla/GER/2010             1
Bat coronavirus (BtCoV/A970/2005)                        1
Betacoronavirus BtCoV/Rhi_hip/Slo48/SLO/2009             1
Miniopterus schreibersii bat coronavirus 1-related       1
Bat coronavirus M.dau/Germany/D8.32/2007                 1
Name: first_taxname, Length: 1284, dtype: int64

In [60]:
df

,Unnamed: 0,Unnamed: 0.1,total_string,date,full_date,first_date,clean_date,taxname,first_taxname
0,0,0,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[[2018], [], [], [], []]",[soloanno],2018,[[2018]],[Alphacoronavirus 1],Alphacoronavirus 1
1,1,1,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[[2018], [], [], [], []]",[soloanno],2018,[[2018]],[Alphacoronavirus 1],Alphacoronavirus 1
2,2,2,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[[2018], [], [], [], []]",[soloanno],2018,[[2018]],[Alphacoronavirus 1],Alphacoronavirus 1
3,3,3,"Seq-entry ::= set {\n class pop-set,\n descr...","[[], [], [24-Jun-2018], [], [], [], [], [], []...","[[24-Jun-2018], [06-Aug-2018]]",2018,"[[24-Jun-2018], [06-Aug-2018]]","[Porcine epidemic diarrhea virus, Porcine epid...",Porcine epidemic diarrhea virus
4,4,4,"Seq-entry ::= set {\n class pop-set,\n descr...","[[], [], [24-Jun-2018], [], [], [], [], [], []...","[[24-Jun-2018], [06-Aug-2018]]",2018,"[[24-Jun-2018], [06-Aug-2018]]","[Porcine epidemic diarrhea virus, Porcine epid...",Porcine epidemic diarrhea virus
...,...,...,...,...,...,...,...,...,...
37173,994,994,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[[1986], [], [], [], [], [], [], [], [], [], [...",[soloanno],1986,[[1986]],[Infectious bronchitis virus],Infectious bronchitis virus
37174,995,995,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[[06-Dec-2004], [], [], [], [], [], [], [], []...",[[06-Dec-2004]],2004,[[06-Dec-2004]],[Infectious bronchitis virus],Infectious bronchitis virus
37175,996,996,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[[07-Nov-2003], [], [], [], [], [], [], [], []...",[[07-Nov-2003]],2003,[[07-Nov-2003]],[Infectious bronchitis virus],Infectious bronchitis virus
37176,997,997,"Seq-entry ::= set {\n class nuc-prot,\n desc...","[[31-Oct-2003], [], [], [], [], [], [], [], []...",[[31-Oct-2003]],2003,[[31-Oct-2003]],[Infectious bronchitis virus],Infectious bronchitis virus


In [109]:
clean_df = df
clean_df = clean_df.drop(['total_string'], axis=1)
clean_df = clean_df.drop(['full_date'], axis=1)
clean_df = clean_df.drop(['date'], axis=1)
clean_df = clean_df.drop(['clean_date'], axis=1)
clean_df = clean_df.drop(['taxname'], axis=1)
clean_df = clean_df.drop(['first_country'], axis=1)
clean_df = clean_df.drop(['affiliations'], axis=1)
clean_df = clean_df.drop(['country'], axis=1)
clean_df = clean_df.drop(['country_clean'], axis=1)

clean_df = clean_df.drop(['last_country'], axis=1)
clean_df = clean_df.drop(['Countries_FL'], axis=1)
clean_df = clean_df.drop(['submission_country'], axis=1)


no_na = clean_df[~clean_df.first_taxname.str.contains("no_taxname")]
no_na = clean_df[~clean_df.first_date.str.contains("no_data")]

no_na.to_excel("vicini_clean_countries_clean.xlsx",sheet_name='foglio1')

In [85]:
clean_df

,Unnamed: 0,Unnamed: 0.1,first_date,first_taxname,affiliations,country,country_clean,first_country,last_country,Countries_FL
0,0,0,2018,Alphacoronavirus 1,"[\n affil ""University of Sao Paul...",[Brasil],[Brasil],Brasil,Brasil,"[Brasil, Brasil]"
1,1,1,2018,Alphacoronavirus 1,"[\n affil ""University of Sao Paul...",[Brasil],[Brasil],Brasil,Brasil,"[Brasil, Brasil]"
2,2,2,2018,Alphacoronavirus 1,"[\n affil ""University of Sao Paul...",[Brasil],[Brasil],Brasil,Brasil,"[Brasil, Brasil]"
3,3,3,2018,Porcine epidemic diarrhea virus,"[\n affil ""Hebei Normal Universit...",[China],[China],China,China,"[China, China]"
4,4,4,2018,Porcine epidemic diarrhea virus,"[\n affil ""Hebei Normal Universit...",[China],[China],China,China,"[China, China]"
...,...,...,...,...,...,...,...,...,...,...
37173,994,994,1986,Infectious bronchitis virus,"[\n affil ""National Veterinary Re...",[South Korea],[South Korea],South Korea,South Korea,"[South Korea, South Korea]"
37174,995,995,2004,Infectious bronchitis virus,"[\n affil ""Konkuk University"",\n ...",[South Korea],[South Korea],South Korea,South Korea,"[South Korea, South Korea]"
37175,996,996,2003,Infectious bronchitis virus,"[\n affil ""Konkuk University"",\n ...",[South Korea],[South Korea],South Korea,South Korea,"[South Korea, South Korea]"
37176,997,997,2003,Infectious bronchitis virus,"[\n affil ""Konkuk University"",\n ...",[South Korea],[South Korea],South Korea,South Korea,"[South Korea, South Korea]"


In [28]:
def extract_sub_country(total_string):
    aff_list = []
    aff = re.findall("(?s)subtype country,(.*?)}", total_string)
    for el in aff:
        aff_list.append(el)

    return aff_list


def clean_sub_country(country):
        clean = re.findall("(?s)name \"(.*?)\"", country)   
        if clean:
            splitter = clean[0].split(":")   
            return splitter[0]
        else:
            return "no_country"

def fill_country(row):
        if row['submission_country_clean'] == "no_country":
            return row['first_country']
        else:
            return row['submission_country_clean']

In [33]:
df['submission_country'] = df.apply(lambda row: extract_sub_country(row['total_string']),axis=1)
df['submission_country_clean'] = df.apply(lambda row: clean_sub_country(extract_first(row['submission_country'])),axis=1)
df['submission_country_clean'] = df.apply(lambda row: fill_country(row),axis=1)

df['submission_country_clean']

0           Colombia
1           Colombia
2           Colombia
3              China
4              China
            ...     
37173    South Korea
37174    South Korea
37175    South Korea
37176    South Korea
37177    South Korea
Name: submission_country_clean, Length: 37178, dtype: object

In [34]:
df['submission_country_clean'].value_ counts()

China                         9911
USA                           2915
Brazil                        1352
Italy                         1282
Japan                         1223
                              ... 
a.ciesielska@nencki.edu.pl       1
Azerbaijan                       1
America                          1
Novavax Inc                      1
qiongyuhao@cdrewu.edu            1
Name: submission_country_clean, Length: 331, dtype: int64

In [105]:
df.loc[(df.submission_country_clean == 'JP'),'submission_country_clean']='Japan'


In [90]:
clean_df = df
clean_df = clean_df.drop(['total_string'], axis=1)

clean_df.to_csv("subcountry.csv",mode='a')

In [3]:
import pandas as pd
df = pd.read_excel('D:/parenti_veri/vicini_clean_countries_clean.xlsx')


In [4]:
 df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,first_date,first_taxname,submission_country_clean
0,0,0,0,2018,Alphacoronavirus 1,Colombia
1,1,1,1,2018,Alphacoronavirus 1,Colombia
2,2,2,2,2018,Alphacoronavirus 1,Colombia
3,3,3,3,2018,Porcine epidemic diarrhea virus,China
4,4,4,4,2018,Porcine epidemic diarrhea virus,China
...,...,...,...,...,...,...
24086,37173,994,994,1986,Infectious bronchitis virus,South Korea
24087,37174,995,995,2004,Infectious bronchitis virus,South Korea
24088,37175,996,996,2003,Infectious bronchitis virus,South Korea
24089,37176,997,997,2003,Infectious bronchitis virus,South Korea


In [10]:
def alphacorona(tax):
    alpha = re.search("(?s)Alphacoronavirus (.*?)", tax)
    if alpha:
        return "Alphacoronavirus"
    else:
        return tax   

In [11]:
import re
df['first_taxname'] = df.apply(lambda row: alphacorona(row['first_taxname']),axis=1)
df['first_taxname']

0                       Alphacoronavirus
1                       Alphacoronavirus
2                       Alphacoronavirus
3        Porcine epidemic diarrhea virus
4        Porcine epidemic diarrhea virus
                      ...               
24086        Infectious bronchitis virus
24087        Infectious bronchitis virus
24088        Infectious bronchitis virus
24089        Infectious bronchitis virus
24090        Infectious bronchitis virus
Name: first_taxname, Length: 24091, dtype: object

In [24]:
df['first_taxname'].value_counts()

Infectious bronchitis virus                             6439
Porcine epidemic diarrhea virus                         4690
Avian coronavirus                                       1259
Human coronavirus OC43                                  1220
Middle East respiratory syndrome-related coronavirus    1189
                                                        ... 
Homo sapiens                                               1
Bat coronavirus NM98-62/GER/2008                           1
Bat coronavirus R.aur/Australia/CoV000/2006                1
Human coronavirus NO/0193/05                               1
Sambar deer coronavirus US/OH-WD388/1994                   1
Name: first_taxname, Length: 587, dtype: int64

In [15]:
def kenyabat(tax):
    kenya = re.search("(?s)Kenya bat coronavirus (.*?)", tax)
    if kenya:
        return "Kenya bat coronavirus"
    else:
        return tax   

In [16]:
df['first_taxname'] = df.apply(lambda row: kenyabat(row['first_taxname']),axis=1)
df['first_taxname']

0                       Alphacoronavirus
1                       Alphacoronavirus
2                       Alphacoronavirus
3        Porcine epidemic diarrhea virus
4        Porcine epidemic diarrhea virus
                      ...               
24086        Infectious bronchitis virus
24087        Infectious bronchitis virus
24088        Infectious bronchitis virus
24089        Infectious bronchitis virus
24090        Infectious bronchitis virus
Name: first_taxname, Length: 24091, dtype: object

In [19]:
def feline(tax):
    kenya = re.search("(?s)Feline coronavirus(.*?)", tax)
    if kenya:
        return "Feline coronavirus"
    else:
        return tax   

In [20]:
df['first_taxname'] = df.apply(lambda row: feline(row['first_taxname']),axis=1)
df['first_taxname']

0                       Alphacoronavirus
1                       Alphacoronavirus
2                       Alphacoronavirus
3        Porcine epidemic diarrhea virus
4        Porcine epidemic diarrhea virus
                      ...               
24086        Infectious bronchitis virus
24087        Infectious bronchitis virus
24088        Infectious bronchitis virus
24089        Infectious bronchitis virus
24090        Infectious bronchitis virus
Name: first_taxname, Length: 24091, dtype: object

In [22]:
def rat(tax):
    kenya = re.search("(?s)Coronavirus RCoV(.*?)", tax)
    if kenya:
        return "Rat coronavirus"
    else:
        return tax   

In [23]:
df['first_taxname'] = df.apply(lambda row: rat(row['first_taxname']),axis=1)
df['first_taxname']

0                       Alphacoronavirus
1                       Alphacoronavirus
2                       Alphacoronavirus
3        Porcine epidemic diarrhea virus
4        Porcine epidemic diarrhea virus
                      ...               
24086        Infectious bronchitis virus
24087        Infectious bronchitis virus
24088        Infectious bronchitis virus
24089        Infectious bronchitis virus
24090        Infectious bronchitis virus
Name: first_taxname, Length: 24091, dtype: object

In [25]:
df.to_excel("vicini_raggruppati.xlsx",sheet_name='foglio1')